# 허프변환

허프변환은 에지가 자잘하게 끊겨 나타나는 경우를 해결할 수 있다.

끊긴 에지를 모아 선분 또는 원 등을 검출할 수 있다.

## 원리
그림은 두 점을 가지고 허프변환의 원리를 설명한다.

(x,y)를 지나는 직선은 y=ax+b 로 표현할 수 있다.

a는 기울기이고, b는 y절편이다.

그림에 있는 빨간 점(x,y)=(1,3)을 대입하면 3=1a+b 인데 정리하면 b=-a+3이다.

이 식에서 b와 a를 변수로 간주하면 그림b의 새로운 공간 (a,b)가 생성된다.

새로운 공간에서 b=-a+3 은 기울기가 -1이고, 절편이 3인 직선이다.

다시 말해 이들 점이 빨간 직선 b=-a+3을 형성한다.

같은 과정을 파란 점 (3,1)에 적용하면 그림b의 파란 직선이 된다.

(a,b)공간에서 두 직선이 만나는 점은 (-1,4)인데 이 점이 그림a의 원래 공간에서 두 점을 지나는 직선의 기울기와 y절편이다.

다시 말해 점선으로 표시한 직선의 방정식은 y=-1x+4이다.

(a,b)공간에서 두 직선이 만나는 점은 투표로 알아낸다.

각각의 직선은 자신이 지나는 점에 1만큼씩 투표를 한다.

결국 그림 b의 경우 직선이 지나지 않는 곳은 0, 직선이 지나는 곳은 1표, 두 직선이 만나는 곳은 2표를 받는다.

허프 변환의 동작을 요약하면 다음과 같다.

입력된 각각의 점 (xi, yi)에 대해 (a,b)공간에 직선 b=-ai+yi를 그린 다음 이들 직선이 만나는 점 (a,b)를 찾아 a를 기울기, b를 y절편으로 취한다.

만나는 점은 투표로 알아낸다.


* 출처 : 컴퓨터비전과딥러닝130~133p(오일석 저)

## 구현
앞에 설명한 허프 변환은 이상적인 상황을 가정했다.

실제 상황에서 구현하려면 몇 가지 사항을 신중하게 고려해야 한다.

### 첫째, 그림에서는 두 점만 고려했는데 현실에서는 많은 점이 있고 점들이 완벽히 일직선을 이루지 못한다.

이 문제는 (a,b)공간을 이산화하여 해결한다.

예를 들어 a와 b의 범위를 -1000~1000으로 설정하고 각각을 크기가 20인 구간 50개로 나누어 칸이 50x50=2500인 2차원 누적 배열 v를 만든다.

v를 0으로 초기화한 다음 각각의 직선은 자신이 지나는 모든 칸에 1만큼씩 투표한다.

### 둘째, 투표가 이뤄진 누적 배열에 잡음이 많다. 그림 2는 가상의 누적 배열을 예시한다.

비최대 억제(non-maximum suppression)는 잡음이 많은 상태에서 한 점을 결정할 때 컴퓨터 비전이 종종 사용하는 방법이다.

캐니 에지에서는 비최대 억제를 사용한 적이 있다.

비최대 억제는 지역 최대(local maximum)가 아닌 점을 억제하고 지역 최대만 남기는 전략이다.

지역 최대 점을 극점(extreme point)이라 부르기도 한다.

그림2의 경우 8-이웃을 사용한다면 색칠한 점이 극점이다.

극점만 남기더라도 잡음이 있어 보통 임곗값을 같이 적용한다.

그림에서는 비최대 억제로 점이 3개 남았는데 임곗값을 5로 설정하면 노란색으로 칠한 점 2개만 최종 선택된다.

### 셋째, 직선 방정식 y=ax+b 를 사용하면 기울기 a가 무한대인 경우 투표가 불가능하다.

이 문제는 극좌표(polar coordinate)에서 직선의 방정식을 표현하는 식1을 도입해서 해결한다.

### 식1 : xsin(세타)+ycos(세타) = 로

허프 변환은 직선의 방정식은 알려주는데 직선의 양 끝점은 알려주지 못한다.

양 끝점을 알아내려면 비최대 억제 과정에서 극점을 형성한 화소를 찾아 가장 먼 곳에 있는 두 화소를 계산하는 추가적인 과정이 필요하다.

허프 변환은 직선뿐 아니라 이론적으로는 어떤 도형이라도 검출할 수 있다.

예를 들어 원을 검출하려면 식2 의 원의 방정식을 이용한다.

이 식은 a,b,r을 포함하므로 (a,b,r)의 3차원 누적 배열을 사용한다.

### 식2 : (x-a)^2 + (y-b)^2 = r^2

# 허프 변환 활용
다음 코드는 원을 검출하는 허프 변환으로 사과 나무 영상에서 사과를 검출하는 프로그램이다.

In [1]:
from google.colab.patches import cv2_imshow
import cv2 as cv

img=cv.imread('apples.jpg')
gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)

오픈 CV 라이브러리를 불러온다.

이미지를 읽는다.

RGB 3차원 이미지를 GRAY 1차원 스케일로 바꾼다.

In [2]:
apples=cv.HoughCircles(gray,cv.HOUGH_GRADIENT,1,200,param1=150,param2=20,minRadius=50,maxRadius=120)

Houghcircles 함수

인수1. 명암 영상에서 원을 검출해 중심과 반지름을 저장한 리스트를 반환한다.

인수2. 여러 변형 알고리즘 중의 하나를 지정하는데 cv.HOUGH_GRADIENT는 에지 방향 정보를 추가로 사용하는 방법이다.

인수3. 누적 배열의 크기를 지정하는데 1로 설정하면 입력 영상과 같은 크기를 사용한다.

인수4. 원 사이의 최소 거리를 지정하는데 작을수록 많은 원이 검출된다.

인수5. 캐니 에지 알고리즘이 사용하는 T_high다.

인수6. 비최대 억제를 적용할 때 쓰는 임곗값이다.

인수7,8. 원의 최소와 최대 반지름을 지정한다.

In [3]:
print(apples)

[[[ 760.5  245.5  118.3]
  [1103.5  224.5  108.4]
  [ 482.5  474.5  119.4]
  [ 223.5  471.5   75.4]
  [ 368.5  668.5  107.3]
  [1005.5  453.5  106.2]
  [ 683.5  506.5   75.3]
  [ 560.5  237.5  105.1]
  [ 344.5  282.5   77.1]
  [ 139.5  654.5  115. ]
  [ 924.5  111.5  119.4]
  [  20.5  462.5  118.3]
  [ 493.5   35.5   79.6]
  [ 907.5  740.5   75.2]
  [1167.5  633.5   62.5]
  [ 710.5   47.5   85.2]
  [ 179.5   94.5   64.9]]]


좌표와 반지름 값

In [4]:
for i in apples[0] :
  cv.circle(img,(int(i[0]),int(i[1])),int(i[2]),(255,0,0),2)

apples 리스트가 가진 원의 중심과 반지름 정보를 이용하여 원래 영상에 원을 그려 넣는다.

In [5]:
cv2_imshow(img)

cv.waitKey()
cv.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.